In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
import glob
import os
import xml.etree.ElementTree as ET

In [11]:
class WiderDataset:
    """
    [left, top, width, height, score]
    """
    def __init__(self,image_folder_path,label_path):
        im_paths = self.get_image_paths(image_folder_path)
        image_labels = self.get_labels(label_path)
        self.data = self.get_image_unified(im_paths,image_labels)
        return 
        
    def get_image_paths(self,folder_path="../dataset/Wider/WIDER_train/images/"):
        folders = glob.glob(folder_path+"*")
        images_paths = []
        for folder in folders:
            images_paths.extend(glob.glob(os.path.join(folder,'*')))
        return images_paths
    
    def get_labels(self,label_file_path):
        label_file = open(label_file_path,'r')
        ## Getting image names
        image_names = []
        flag = False
        image_labels = {}
        for line in label_file:
            sp = line.strip().split('/')
            if len(sp)>1:
                image_names.append(sp[-1])
                image_labels[image_names[-1]] = []
            cor = line.strip().split(' ')
            if len(cor)==1:
                continue
            image_labels[image_names[-1]].append([int(val) for val in cor[:4]])
        label_file.close()
        return image_labels
        
    def get_image_unified(self,image_paths,image_lables_dict):
        data = []
        for im_p in image_paths:
            im_name = im_p.strip().split('/')[-1]
            data_object = {}
            data_object['path'] = im_p
            data_object['bbox'] = image_lables_dict[im_name]
            data.append(data_object)
        return data
    
    def get_item(self,i,style='anchor'):
        image = np.array(Image.open(self.data[i]['path']))
        label = self.data[i]['bbox']
        if style=='anchor':
            label = self.get_anchor_type(label)
        ### do the preprocessing on the image here
        ## TODO 
        # image = self.preprocess(image)
        return image,label
    
    def get_anchor_type(self,labels):
        new_label = []
        for l in labels:
            new_label.append([l[0],l[1],l[0]+l[2],l[1]+l[3]])
        return new_label
        
    
    def preprocess(self,image):
        return image

In [12]:
image_folder_path = "../../dataset/Wider/WIDER_train/images/"
label_path = "../../dataset/Wider/wider_face_split/wider_face_train_bbx_gt.txt"

In [13]:
widerDataset = WiderDataset(image_folder_path,label_path)

In [14]:
data = widerDataset.data

In [15]:
data[0]

{'path': '../../dataset/Wider/WIDER_train/images/30--Surgeons/30_Surgeons_Surgeons_30_568.jpg',
 'bbox': [[356, 74, 90, 132]]}

In [16]:
x = widerDataset.get_item(0)

In [17]:
x[0].shape

(768, 1024, 3)

In [18]:
x = ['stride1','stride2']
y = [[1,23,3],[4,5,6]]
z = dict(zip(x,y))

In [19]:
z

{'stride1': [1, 23, 3], 'stride2': [4, 5, 6]}

# Dataset class for our dataset

In [23]:
class maviData:
    """
    [left, top, width, height, score]
    each object : {'bbox': ,'path':}
    """
    def __init__(self,image_root_dir,annotations_root_dir):
        im_paths = glob.glob(image_root_dir+"*")
        image_labels = self.get_labels(annotations_root_dir)
        self.data = self.get_image_unified(im_paths,image_labels)
        return
        
    def get_labels(self,annotations_root_dir):
        ann_locs = glob.glob(annotations_root_dir+"*")
        image_labels ={}
        for ann in ann_locs:
            bboxs = self.get_topLeftWidthHeight(ann)
            im = self.get_last(ann)[:-4]
            image_labels[im] = bboxs
        return image_labels
    
    def get_image_unified(self,im_paths,image_labels):
        data = []
        for path in im_paths:
#             print(path)
            dataObject = {}
            dataObject['path'] = path
            dataObject['bbox'] = image_labels[self.get_last(path)[:-4]]
            if dataObject['bbox']:
                data.append(dataObject)
        return data
        
    def get_item(self,i,style='anchor'):
        image = np.array(Image.open(self.data[i]['path']))
        label = self.data[i]['bbox']
        if style=='anchor':
            label = self.get_anchor_type(label)
        ### do the preprocessing on the image here
        ## TODO 
        # image = self.preprocess(image)
        return image,label
    
    def get_anchor_type(self,labels):
        new_label = []
        for l in labels:
            new_label.append([l[0],l[1],l[0]+l[2],l[1]+l[3]])
        return new_label
        
    
    def preprocess(self,image):
        return image
    
    def get_topLeftWidthHeight(self,filepath):
        tree = ET.parse(filepath)
        root = tree.getroot()
        annotations = []
        for bndbox in root.findall('object/bndbox'):
            xmin = bndbox.find('xmin').text
            xmax = bndbox.find('xmax').text
            ymin = bndbox.find('ymin').text
            ymax = bndbox.find('ymax').text
            x = xmin
            y = ymin 
            w = str(int(xmax) - int(xmin))
            h = str(int(ymax) - int(ymin))
            annotations.append((int(x),int(y),int(w),int(h)))
        return annotations
    
    def get_last(self,x):
        a = x.split('/')[-1]
        return a

In [24]:
image_loc = "../../dataset/images/"
annotation_loc = "../../dataset/annotations/"

In [25]:
mData = maviData(image_loc,annotation_loc)

In [26]:
data = mData.data

In [27]:
mData.data[0]

{'path': '../../dataset/images/2019-06-17 15_01_25__018426__44841b92-90d1-11e9-8b8d-dd1eda89fb93.jpg',
 'bbox': [(447, 156, 49, 59)]}

In [35]:
len(data)

1759

In [36]:
import random
import shutil

## splitting for test and train 20:80

In [38]:
# impaths = glob.glob(image_loc+"*")

In [40]:
# random.shuffle(impaths)

In [44]:
# impaths_train = impaths[:int(len(impaths)*0.8)-1]

In [45]:
# len(impaths_train)

1406

In [46]:
# impaths_test = impaths[int(len(impaths)*0.8)-1:]

In [47]:
# len(impaths_test)

353

In [52]:
# train_loc = "../../dataset/train_split_images/"
# test_loc = "../../dataset/test_split_images/"

In [53]:
# for i in impaths_train:
#     shutil.copy2(i,train_loc)
# for j in impaths_test:
#     shutil.copy2(j,test_loc)